In [ ]:
from itertools import compress
import pandas as pd
from operator import itemgetter
from itertools import groupby
import json
from bert_serving.client import BertClient
import os.path
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.autograd import Variable  
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image  
import ftplib
import time
import threading

#1.加载训练文件
with open("/data/fjs/data/math/math.json",'r') as load_f:
    allItems = json.load(load_f)#list中条目是dict
    
#2.探索性数据分析EDA
#2.1 习题难度分布
diffs={}#统计难度类别
for item in allItems:
    diff = item['qdiff'] #获取习题难度
    diffs[diff]=diffs.get(diff,0)+1 #统计每个元素出现的个数。
#{0: 9, 1: 5434, 2: 198597, 3: 482016, 4: 212266, 5: 1678}
df = pd.DataFrame(diffs,index=[0])
df.plot(kind='bar')
plt.show()
#2.2 习题概念分布
cons={}#统计概念类别
for item in allItems:
    con =item['qklg']
    conp = con.split(',')#逗号分隔
    for p in conp:#具体概念点
        cons[p]=cons.get(p,0)+1
df = pd.DataFrame(list(cons.items()))
print (df.shape[0])#知识点数
print (np.mean(df.loc[:,1]))#知识点数的均值
#绘制箱形图
def drawBox(freqs):
    #创建箱形图
    #第一个参数为待绘制的定量数据
    #第二个参数为数据的文字说明
    plt.boxplot([freqs], labels=['Concepts'])
    plt.title('Frequecy Of Concpets')
    plt.show()
drawBox(df.loc[:,1])

#3.习题的文本、概念基于BERT模型Embedding；习题的图像基于Resnet模型提取视觉特征
f = ftplib.FTP("10.21.3.24")  # 实例化FTP对象
f.login("ftpuser", "dm2019")  # 登录
def ftp_download(remotename,localname):
    bufsize = 1024  # 设置缓冲器大小
    fp = open(localname, 'wb')
    f.retrbinary('RETR %s' % remotename, fp.write, bufsize)
    fp.close()
#预训练resnet50模型
transform1 = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
resnet50_feature_extractor = models.resnet50(pretrained = True)
resnet50_feature_extractor.fc = nn.Linear(2048, 2048)
torch.nn.init.eye(resnet50_feature_extractor.fc.weight)
for param in resnet50_feature_extractor.parameters():
    param.requires_grad = False
#定义18个线程提取特征，每个线程提取5万条
def threadFunction(n,items):
    print ("Start Threading %d."%n)
    start = time.time()
    VItems = []#保存列表
    for item in items:
        VItem={}#dict
        VItem['quid']=item['quid']
        VItem['qdiff']=item['qdiff']
        #3.1 文本、概念embedding
        texts=item['qtext']#list
        strtext=''
        for text in texts:
            strtext=strtext+text
        concepts=item['qklg']
        bc = BertClient()#连接BERT Server
        VItem['qtext']=bc.encode([strtext])[0]#list,768长度
        VItem['qklg']=bc.encode([concepts])[0]#list，768长度
        #3.2 图片视觉特征提取
        images=[]
        for imgftppath in item['qimage']:
            imgftppath = imgftppath[17:]#截掉ftp://10.21.3.24/
            imglocalpath = "/data/fjs/data/imgs/"+imgftppath.split("/")[-1]
            ftp_download(imgftppath,imglocalpath)#从ftp下载到本地
            image = Image.open(imglocalpath)#打开本地图片
            images.append(image)
        #多图拼接成一张长图
        if len(images)==0:
            VItem['qimage']=np.zeros(2048)
        else:
            width,height=images[0].size
            for image in images:
                w,h = image.size
                if w>width:width=w
                if h>height: height=h
            longImg =  Image.new(images[0].mode,(width,height*len(images)))
            for i,im in enumerate(images):
                longImg.paste(im,box=(0,i*height))
            #长图向量化    
            imgarr = np.array(longImg)
            if imgarr.shape[2] == 4: #四通道转为三通道
                img1 = longImg.convert("RGB")
            #img = Image.fromarray(img.astype('uint8')).convert('RGB')
            img2 = transform1(img1)
            x = Variable(torch.unsqueeze(img2, dim=0).float(), requires_grad=False)
            y = resnet50_feature_extractor(x)
            y = y.data.numpy()
            VItem['qimage']=y #2048长度
        VItems.append(VItem)
    with open("/data/fjs/data/math/mathV"+str(n)+".json","w") as dump_f:
        json.dump(VItems,dump_f)#保存到json文件
    print ('file %d saved.'%n)
    end = time.time()
    print ("Complete time: %f s" % (end - start))

#将全部90万条记录分成18份提取特征
for i in range(0,len(allItems),50000):
    mThread = threading.Thread(target=threadFunction, args=(i,allItems[i:i+50000]))
    mThread.start()

/usr/local/lib/python3.5/dist-packages/torchvision-0.2.1-py3.5.egg/torchvision/transforms/transforms.py:200: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [34]:
import os.path
import json 
import ftplib
from pathlib import Path
#1.加载训练文件
with open("/data/fjs/data/math/math.json",'r') as load_f:
    allItems = json.load(load_f)#list中条目是dict
    
#count =0
f = ftplib.FTP("10.21.3.24")  # 实例化FTP对象
f.login("ftpuser", "dm2019")  # 登录
def ftp_download(remotename,localname):
    bufsize = 1024  # 设置缓冲器大小
    fp = open(localname, 'wb')
    f.retrbinary('RETR %s' % remotename, fp.write, bufsize)
    fp.close()
#将ftp全部手动在操作系统上下载到本地，然后再调整异常的，减少磁盘IO和网络传输时间
imgs=[]
for item in allItems:
    for img in item['qimage']:
        #if img.find('ftp://10.21.3.24/')==-1:print (img)
        #if img.find('upimages')==-1:print (img)
        ipath = img[img.find('upimages'):]#截取ftp://10.21.3.24/tiku/images/后的目录
        ipath = "/data/fjs/data/mathimgs/"+ ipath
        #if img.find('\\')>=0:img = img[:-1] #去除最后一个字符
        if ipath.find('\\')>=0:ipath = ipath[:-1] #去除最后一个字符
        if not os.path.exists(ipath):#本地不存在，去下载。
            img = img[17:]#截掉ftp://10.21.3.24/
            imgs.append(img)
for img in imgs:
    ipath = img[img.find('upimages'):]
    ipath = "/data/fjs/data/mathimgs/"+ ipath
    ftp_download(img,ipath)#从ftp下载到本地 
    print (ipath)

In [43]:
import json
import pandas as pd
#将list中dict的json文件转化为DataFrame并保存为CSV，可以利用python的矩阵计算优势，而不是采用循环
with open("/data/fjs/data/math/math.json",'r') as load_f:
    allItems = json.load(load_f)#list中条目是dict
csvlist=[]
for item in allItems:
    strtext=''
    for text in item['qtext']: strtext=strtext+text
    strimage=''
    for image in item['qimage']:strimage=strimage+","+image
    tmplist=[item['quid'],item['qdiff'],item['qklg'], strtext, strimage]
    csvlist.append(tmplist)
df = pd.DataFrame(csvlist, columns=['quid', 'qdiff', 'qklg','qtext','qimage']) 
print (df.shape)
df.to_csv('/data/fjs/data/math/math.csv',index=False,sep='|')

(900000, 5)


In [12]:
#去除空置，在多模态学习上更有意义。
import pandas as pd
data = pd.read_csv("/data/fjs/data/math/math.csv",sep='|') #print (data.head())
data=data.dropna(axis=0,how='any')#删除包含空置的行，主要是没有图片。
#data = data.fillna('none')
print (data.shape)
#print (data.head())
data.to_csv('/data/fjs/data/math/mathmin.csv',index=False,sep='|')

(234921, 5)


In [3]:
import pandas as pd
import numpy as np
import json
from bert_serving.client import BertClient
import torch
import torch.nn as nn
from torchvision import models,transforms
from torch.autograd import Variable
from PIL import Image
import time

#1.预训练resnet50模型构建
transform1 = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
resnet50_feature_extractor = models.resnet50(pretrained = True)
resnet50_feature_extractor.fc = nn.Linear(2048, 2048)
torch.nn.init.eye(resnet50_feature_extractor.fc.weight)
for param in resnet50_feature_extractor.parameters():
    param.requires_grad = False
    
#2.连接BertService
bc = BertClient()#连接BERT Server

#3.多图拼接长图并向量化函数定义
def fn_ImageToVec(x):
    images=[]
    for img in x.split(",")[1:]:#ftp图片用逗号隔开
        ipath = img[img.find('upimages'):]#截取ftp://10.21.3.24/tiku/images/后的目录
        ipath = "/data/fjs/data/mathimgs/"+ ipath
        image = Image.open(ipath)#打开本地图片
        images.append(image)
    #多图拼接成一张长图
    width,height=images[0].size
    for image in images:
        w,h = image.size
        if w>width:width=w
        if h>height: height=h
    longImg =  Image.new(images[0].mode,(width,height*len(images)))
    for i,im in enumerate(images):
        longImg.paste(im,box=(0,i*height))
    #长图向量化    
    imgarr = np.array(longImg)
    if imgarr.shape[2] == 4: #四通道转为三通道
        longImg = longImg.convert("RGB")
    longImg = transform1(longImg)
    x = Variable(torch.unsqueeze(longImg, dim=0).float(), requires_grad=False)
    y = resnet50_feature_extractor(x)
    return y.data.numpy() #2048长度

start = time.time()    
#4.加载训练文件
data = pd.read_csv("/data/fjs/data/math/mathmin.csv",sep='|') 
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#data = data.head()
#5.1概念向量化
vecklg = bc.encode(data['qklg'].tolist())
data = pd.concat([data,pd.DataFrame(vecklg)],axis=1)#768维
data=data.drop(['qklg'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#5.2文本向量化
vectxt = data['qtext'].apply(lambda x: bc.encode([x])[0])
data = pd.concat([data,pd.DataFrame(vectxt.tolist())],axis=1)#768维
data=data.drop(['qtext'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#5.3 图像向量化
npvimg=data['qimage'].apply(lambda x: fn_ImageToVec(x)[0])
data = pd.concat([data,pd.DataFrame(npvimg.tolist())],axis=1)#2048维
data=data.drop(['qimage'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#6.保存向量化后的文件
data.to_csv("/data/fjs/data/math/mathV.csv",index=False,sep='|')
end = time.time()
print ("Complete time: %f s" % (end - start))

/usr/local/lib/python3.5/dist-packages/torchvision-0.2.1-py3.5.egg/torchvision/transforms/transforms.py:200: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.eye is now deprecated in favor of nn.init.eye_.


234921 rows and 5 cols
5 rows and 772 cols


/home/fjs/.local/lib/python3.5/site-packages/bert_serving/client/__init__.py:277: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


5 rows and 1539 cols
5 rows and 3586 cols
Complete time: 6.398251 s


In [ ]:
import pandas as pd
import numpy as np
import json
from bert_serving.client import BertClient
import torch
import torch.nn as nn
from torchvision import models,transforms
from torch.autograd import Variable
from PIL import Image
import time

#1.预训练resnet50模型构建
transform1 = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
resnet50_feature_extractor = models.resnet50(pretrained = True)
resnet50_feature_extractor.fc = nn.Linear(2048, 2048)
torch.nn.init.eye(resnet50_feature_extractor.fc.weight)
for param in resnet50_feature_extractor.parameters():
    param.requires_grad = False
    
#2.连接BertService
bc = BertClient()#连接BERT Server

#3.多图拼接长图并向量化函数定义
def fn_ImageToVec(x):
    images=[]
    for img in x.split(",")[1:]:#ftp图片用逗号隔开
        ipath = img[img.find('upimages'):]#截取ftp://10.21.3.24/tiku/images/后的目录
        ipath = "/data/fjs/data/mathimgs/"+ ipath
        image = Image.open(ipath)#打开本地图片
        images.append(image)
    #多图拼接成一张长图
    width,height=images[0].size
    for image in images:
        w,h = image.size
        if w>width:width=w
        if h>height: height=h
    longImg =  Image.new(images[0].mode,(width,height*len(images)))
    for i,im in enumerate(images):
        longImg.paste(im,box=(0,i*height))
    #长图向量化    
    imgarr = np.array(longImg)
    if imgarr.shape[2] == 4: #四通道转为三通道
        longImg = longImg.convert("RGB")
    longImg = transform1(longImg)
    x = Variable(torch.unsqueeze(longImg, dim=0).float(), requires_grad=False)
    y = resnet50_feature_extractor(x)
    return y.data.numpy() #2048长度

start = time.time()    
#4.加载训练文件
data = pd.read_csv("/data/fjs/data/math/mathmin.csv",sep='|') 
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#5.1概念向量化
vecklg = bc.encode(data['qklg'].tolist())
data = pd.concat([data,pd.DataFrame(vecklg)],axis=1)#768维
data=data.drop(['qklg'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#5.2文本向量化
vectxt = data['qtext'].apply(lambda x: bc.encode([x])[0])
data = pd.concat([data,pd.DataFrame(vectxt.tolist())],axis=1)#768维
data=data.drop(['qtext'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#5.3 图像向量化
npvimg=data['qimage'].apply(lambda x: fn_ImageToVec(x)[0])
data = pd.concat([data,pd.DataFrame(npvimg.tolist())],axis=1)#2048维
data=data.drop(['qimage'], axis=1)
print ("%d rows and %d cols"%(data.shape[0],data.shape[1]))
#6.保存向量化后的文件
data.to_csv("/data/fjs/data/math/mathV.csv",index=False,sep='|')
end = time.time()
print ("Complete time: %f s" % (end - start))

/usr/local/lib/python3.5/dist-packages/torchvision-0.2.1-py3.5.egg/torchvision/transforms/transforms.py:200: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.eye is now deprecated in favor of nn.init.eye_.


234921 rows and 5 cols


/home/fjs/.local/lib/python3.5/site-packages/bert_serving/client/__init__.py:277: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '
